## NumPy Monte Carlo

Take an hour or so to discuss with your teammates what changes you can make to your code to take advantage of the features of NumPy arrays. For this portion, take the first 30 minutes to think about this by yourself, then discuss for half an hour with your group. We will tell you one option for rewriting your code at the end of this discussion period.

Consider each function in our code - which ones could benefit from using numpy arrays? Consider each function and include thoughts for each one in your blog post for today.

## np calculate_distance

In [1]:
import math
def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two 3D coordinates.
    Parameters
    ----------
    coord1, coord2: list
        The atomic coordinates
    box_length : float
        The box length. If given, the minimum image convention will be used to calculate the distance.
    Returns
    -------
    distance: float
        The distance between the two points.
    """
    distance = 0
    for i in range(3):
        dim_dist = (coord1[i] - coord2[i])
        if box_length:
            coord_dist = dim_dist - box_length * np.round(dim_dist / box_length)
        dim_dist = dim_dist**2
        distance += dim_dist
    distance = np.sqrt(distance)
    return distance


import numpy as np

point1 = np.array([0,0,0])
point2 = np.array([0,8,0])


calculate_distance(point1, point2)
dimensional_distance = point2 - point1
print(dimensional_distance)


dimensional_distance = point1 - point2
print(dimensional_distance)


dd2 = dimensional_distance ** 2 
dd2_sum = dd2.sum()
distance = math.sqrt(dd2_sum)

print(distance)


import math
def calculate_distance_np(coord1, coord2, box_length=None):
    """
    Calculate the distance between two 3D coordinates.
    Parameters
    ----------
    coord1, coord2: np.array
        The atomic coordinates
    
    box_length : float
        The box length. If given, the minimum image convention will be used to calculate the distance.
    Returns
    -------
    distance: float
        The distance between the two points.
    """
    
    coord_dist = coord1 - coord2
    
    
    if box_length:
            coord_dist = coord_dist - box_length * np.round(coord_dist / box_length)
    
   

#how many axis it has 

    if coord_dist.ndim < 2:
        coord_dist = coord_dist.reshape(1,-1)


    
    
    
    
    
    coord_dist = coord_dist ** 2
    coord_dist_sum = coord_dist.sum(axis=1)
    distance = np.sqrt(coord_dist_sum)
    return distance



## Check your funtction w/assert statements

point_1 = np.array([[0,0,0]])
point_2 = np.array([[1,0,0],[0,1,1]])
dist1 = calculate_distance_np(point_1, point_2)
assert dist1[0] == 1
assert dist1[1] == math.sqrt(2)

point_1 = np.array([0,0,0])
point_2 = np.array([0,1,1])
dist2 = calculate_distance_np(point_1, point_2)

assert dist2 == math.sqrt(2)
coord_set1 = np.array([[0, 8, 0], [0, 1.5, 0]])
coord_set2 = np.array([[0, 8, 0], [0, 1.5, 0]])


calculate_distance_np(coord_set1, coord_set2, 10)
coord_set1.ndim

[0 8 0]
[ 0 -8  0]
8.0


2

## Pair Energy Numpy

In [2]:
def calculate_pair_energy_np(coordinates, i_particle, box_length, cutoff):
    
    """
    
    Calculate interaction eenrgy of particle w/ its environment (all other particles in sys)
    
    Parameters
    ----------------
    coordinates : list
        the coordinates for all particles in sys
    i_particle : int
        particle number for which to calculate energy
        
    cutoff : float
        simulation cutoff. beyond distnaces, interactions aren't calculated
    
    box length : float
    
        length of simultion box. assumse cubic boc
        
    
    Returns
    ---------------
    
    float
        pairwise interaction energy of ith particle w/all other particles in sys
        
    
    """

    particle_distances  = calculate_distance_np(coordinates[i_particle], coordinates[i_particle+1:], box_length) 
    particle_distances_filtered = particle_distances[particle_distances < cutoff]
    return calculate_LJ(particle_distances_filtered).sum()


## total energy np

In [7]:
import math

import random

import numpy as np


import os 


def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    
    coordinates = np.genfromtxt(filepath,skip_header=2,usecols=[1,2,3])

    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
          
    
    
        
    return coordinates, box_length



print(os.getcwd())
config1_file = os.path.join("../../lj_sample_configurations", "lj_sample_config_periodic1.txt") 

sample_coords, box_length = read_xyz(config1_file) 

print(sample_coords[0])




def calculate_total_energy_np(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : list
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    num_atoms = len(coordinates)

    
    pair_energies = np.array([calculate_pair_energy_np(coordinates,i,box_length,cutoff) for i in range(num_atoms)])
    
    print(pair_energies.sum())
    return pair_energies.sum()
    


def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    
    r6_term = np.power(1/r_ij, 6)
    r12_term = np.power(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    return pairwise_energy


/home/joyce/chem_280/team1-project/homework/day_4
[-0.11263626  1.38509308 -0.88420351]


In [8]:

sum = calculate_total_energy_np(sample_coords, box_length, 3) 



-4351.540194543866


## tail correction

In [9]:
def calculate_tail_correction(n,b,r_c):
    r3_term = np.power(1/r_c, 3)
    r9_term = (1/3) * (np.power(r3_term, 9))
    NV_term = (8*math.pi/3) * ((n**2)/(b**3))
    tail_correction = NV_term * (r9_term - r3_term)
    
    return tail_correction

## accept or reject function

In [10]:
def accept_or_reject(delta_U, beta):
    """
    
    
    
    
    
    Accept or reject a move based on metropolis criterion.
    
    Paramaters
    ----------------
    delta_U : float
    
        change in energy for moving systems from m to n
    
    
    
    beta : float
        1/temperature
    
    
    
    Returns
    --------
    bool
        whether move is accepted-true
        
    
        
    
    
    
    
    
    """
    
    
    if delta_U <= 0.0:
        accept = True
    else:
        #Gen random number on(0,1)
        random_number = random.random()
        p_acc = math.exp(-beta*delta_U)
        
        if random_number < p_acc:
            accept = True
         
        else:
            accept = False
            
    return accept

## monte carlo np

In [12]:
import numpy as np

import random 


#Read or generate inital coordiantes
coordinates, box_length = read_xyz('../../lj_sample_configurations/lj_sample_config_periodic1.txt')

# Set simulation parameters
reduced_temperature = 0.9

num_steps = 5000

max_displacement = 0.1

cutoff = 3

freq = 1000

# Calculated quantities

beta = 1 / reduced_temperature

num_particles = len(coordinates)

# Energy calculations 

total_energy = calculate_total_energy_np(coordinates, box_length, cutoff)

print(total_energy)

tail_correction = calculate_tail_correction(num_particles, box_length, cutoff)

print(tail_correction)

total_energy += tail_correction

for step in range(num_steps):
        
        # 1. Randomly pick one of particles
        random_particle = random.randrange(num_particles)
        
        # 2. Calculate in interavtion energy of sleectd particle w/system and store this value
        current_energy = calculate_pair_energy_np(coordinates, random_particle, box_length, cutoff)
       
        # 3. Gnerate random x,y,z displacement
        
        xyz_rand = np.random.uniform(-max_displacement, max_displacement, [1,3])

        



        #4. Modify coordinate of Nth particle by genrated displacements
        
        
        coordinates[random_particle] = coordinates[random_particle] + xyz_rand

        
        #5. Calculate interaction energy of moved particle w/system
        proposed_energy = calculate_pair_energy_np(coordinates, random_particle, box_length, cutoff)
        
        delta_energy = proposed_energy - current_energy
        
        #6. Calculate if we accept move based on energy diff
        
        accept = accept_or_reject(delta_energy, beta)
        #7. if accept, move particle
        if accept: 
            total_energy += delta_energy
        else:
            # Move is not accepted, roll back coordinates
            coordinates[random_particle] = coordinates[random_particle] - xyz_rand



        #8. print energy if step is multiple of freq
        
        if step % freq == 0:
            print(step, total_energy/num_particles)

-4351.540194543866
-4351.540194543866
-198.57968378223148
0 -5.687731720288053
1000 -5.794303727726162
2000 -6.152994184263812
3000 -6.646894026972656
4000 -7.380887999886412


In [13]:
xyz_rand = np.random.uniform(-max_displacement, max_displacement, 3)
print(xyz_rand)

[ 0.02103187 -0.04365282 -0.07924593]
